!pip install yfinance


### Data Cleaning and Preprocessing

##### Exploratory Data Analysis (EDA)
- Check the structure of the dataset:
    - Verify column names and data types
- Identify missing values:
    - Drop weekends as they are non trading days.
    - If missing values exist in **Open/Close** and **High/Low**: Fill with the average of previous and next day's values. 
    - If missing values exist in **Volume**: Fill with the median.
- Identify duplicate rows:
  - If duplicate trading days exist for the same stock, drop them.

##### Split dataset into training, test, and validation set
- Use **70/15/15 split** for training, testing, and validation


##### Feature Scaling 
- Scale features using **StandardScaler**

In [21]:
import numpy as np
import pandas as pd
import yfinance as yf

ModuleNotFoundError: No module named 'yfinance'

In [12]:
# Load the CSV file
stocks = pd.read_csv("../data_files/stocks.csv")

# Display basic info and first few rows
print(stocks.columns)
stocks.info() 
stocks.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data_files/stocks.csv'

In [13]:
print("Data Types:",stocks.dtypes )
print("Missing Values:", stocks.isnull().sum())

NameError: name 'stocks' is not defined

Each stock has seperate columns for Close, High, Low, Open, and Volume. There are many NaN values in the dataset for stocks that did not exit or trade on a specific date. 

In [14]:
#Deal with missing values

#Stock market closed on weekends so drop weekends
stocks['Date_'] = pd.to_datetime(stocks['Date_'])
stocks = stocks.loc[stocks['Date_'].dt.dayofweek < 5]
#https://gpttutorpro.com/pandas-dataframe-filtering-using-datetime-methods/

#Replace missing values in Open,Close, High, and Low columns
for col in stocks.columns:
    if "Open" in col or "Close" in col or "High" in col or "Low" in col:
        # Find the first valid index where trading starts
        first_valid_index = stocks[col].first_valid_index()

        if first_valid_index is not None:
            # Fill missing values only after the stock starts trading with average of previous and next day's values.
            stocks.loc[first_valid_index:,col] = stocks.loc[first_valid_index:,col].fillna((stocks[col].shift(1)+stocks[col].shift(-1))/2)

            # Forward-fill and backward-fill only after the first valid trading day (fills with closest)
            stocks.loc[first_valid_index:,col] = stocks.loc[first_valid_index:,col].ffill().bfill()

#https://medium.com/@farisyid/penggunaan-ffill-dan-bfill-pada-proses-data-cleaning-b4f3bfec9767#:~:text='ffill'%20which%20means%20forward%20fill%20and%20'bfill',such%20as%20DataFrame%20or%20Series%20in%20Pandas.&text=Instead%2C%20the%20'bfill'%20method%20fills%20the%20missing,the%20missing%20value%20in%20the%20data%20sequence.

#replace missing values in volume columns with the median
volume_cols = []
for col in stocks.columns:
    if "Volume" in col:
        volume_cols.append(col)
for col in volume_cols:
    stocks.loc[:, col] = stocks[col].fillna(stocks[col].median())

stocks

NameError: name 'stocks' is not defined

Missing values in the dataset were handled by first removing weekends, as they are non-trading days. For Open, Close, High, and Low prices, missing values were filled using the average of the previous and next trading day's values, ensuring that data was only adjusted after the stock had begun trading. Volume data was completed using the median to maintain consistency and avoid skewing results. This approach ensures realistic stock data representation.

In [15]:
#check if duplicate rows exist
print("Number of duplicate rows:", stocks.duplicated().sum())


NameError: name 'stocks' is not defined

No duplicate rows.

In [16]:
stocks = stocks.drop(['Unnamed: 0', 'Date_'], axis = 1)

NameError: name 'stocks' is not defined

In [17]:
def fetch_stock_data(ticker, start='2020-01-01', end='2024-10-01'):
    data = yf.download(ticker, start=start, end=end)

    if isinstance(data.columns, pd.MultiIndex):
        data.columns = [col[0] for col in data.columns]

    data['Return'] = data['Close'].pct_change()

    data['Forward_Return'] = data['Return'].shift(-1)

    data['Target'] = np.where(data['Forward_Return'] > 0.01, 2,  # Buy
                             np.where(data['Forward_Return'] < -0.01, 0,  # Sell
                                     1))  # Hold
    data.dropna(inplace=True)

    print(f"Class distribution: {data['Target'].value_counts()}")
    
    return data[['Close', 'Volume', 'Return', 'Target']]

In [18]:
google = fetch_stock_data('GOOG')

NameError: name 'yf' is not defined

In [19]:
google.head()

NameError: name 'google' is not defined

### Logistic Regression

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# ---------------------------
# Logistic Regression Setup
# ---------------------------

# Create the target variable:
# Predict if Close_AAPL increases (1) or decreases (0) the next day.
#stocks["Target"] = (stocks["Close_AAPL"].shift(-1) > stocks["Close_AAPL"]).astype(int)

# Drop the last row (no "next day" available)
#stocks = stocks[:-1]

# Remove the Date column (non-numeric)
#if "Date" in stocks.columns:
#    stocks = stocks.drop(columns=["Date_"])

# Define features (X) and target (y)
X = google.drop(columns=["Target"])
y = google["Target"]

# ----- Additional Safeguard: Impute any remaining missing values in features -----
# This ensures that even if some NaNs were missed during cleaning, they are filled.
X = X.fillna(X.mean())

# Confirm no NaNs remain
assert X.isnull().sum().sum() == 0, "There are still missing values in the features!"

# ---------------------------
# Split the Data
# ---------------------------
# Use a 70/15/15 split for training, validation, and testing.
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

# ---------------------------
# Feature Scaling
# ---------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

# ---------------------------
# Train Logistic Regression Model
# ---------------------------
log_reg = LogisticRegression(multi_class = 'multinomial', max_iter=500, random_state=42, class_weight = 'balanced')
log_reg.fit(X_train_scaled, y_train)

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay

# ---------------------------
# 1. Plot the ROC Curve
# ---------------------------
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score
from itertools import cycle

# Get predicted probabilities for the positive class on the test set.
# Binarize y for ROC AUC score (e.g., one-vs-rest style)
y_test_bin = label_binarize(y_test, classes=[0, 1, 2])
y_score = log_reg.predict_proba(X_test_scaled)
n_classes = y_test_bin.shape[1]

roc_auc = roc_auc_score(y_test_bin, y_score, average='macro', multi_class='ovr')

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

macro_roc_auc = roc_auc_score(y_test_bin, y_score, average='macro', multi_class='ovr')
print(f"Multiclass ROC AUC (macro-average): {macro_roc_auc:.3f}")

colors = cycle(['blue', 'orange', 'green'])
plt.figure(figsize=(10, 8))

for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f"Class {i} (AUC = {roc_auc[i]:.2f})")

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curve - Logistic Regression')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

# ---------------------------
# 2. Plot the Logistic Regression Coefficients
# ---------------------------
# Retrieve the coefficients and corresponding feature names.
coef = log_reg.coef_[0]
feature_names = X.columns

# Create a DataFrame and sort by absolute coefficient values.
coef_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coef})
coef_df['AbsCoefficient'] = coef_df['Coefficient'].abs()
coef_df = coef_df.sort_values(by='AbsCoefficient', ascending=False)

# Plot the top 20 features.
plt.figure(figsize=(10, 8))
plt.barh(coef_df['Feature'].head(20)[::-1], coef_df['Coefficient'].head(20)[::-1])
plt.xlabel('Coefficient Value')
plt.title('Top 20 Logistic Regression Coefficients')
plt.show()

# ---------------------------
# 3. Plot the Confusion Matrix
# ---------------------------
cm = confusion_matrix(y_test, log_reg.predict(X_test_scaled))
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix for Logistic Regression Model')
plt.show()



NameError: name 'google' is not defined

In [ ]:
# ---------------------------
# Evaluate the Model
# ---------------------------

# Evaluate on Validation Set
y_val_pred = log_reg.predict(X_val_scaled)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_report = classification_report(y_val, y_val_pred)

print("Validation Accuracy:", val_accuracy)
print("Validation Classification Report:\n", val_report)

# Evaluate on Test Set
y_test_pred = log_reg.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_report = classification_report(y_test, y_test_pred)

print("Test Accuracy:", test_accuracy)
print("Test Classification Report:\n", test_report)

### Neural Network Model:

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.utils import class_weight

In [ ]:
def build_multiclass_model(input_shape):
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=(input_shape,)),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu'),
        layers.Dense(3, activation='softmax')  # 3 output neurons for 3 classes with softmax activation
    ])
    
    # Use categorical_crossentropy for multi-class classification
    model.compile(optimizer='adam', 
                 loss='sparse_categorical_crossentropy',  # Use sparse if target is not one-hot encoded
                 metrics=['accuracy'])
    return model
    
model = build_multiclass_model(X_train.shape[1])

# Calculate class weights
weights = class_weight.compute_class_weight(class_weight='balanced', 
                                           classes=np.unique(y_train), 
                                           y=y_train)
class_weights = dict(enumerate(weights))

# Fit with class weights
history = model.fit(
    X_train_scaled, y_train,  # Using integer labels directly
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    class_weight=class_weights,
    verbose=1
)

# Evaluate on test data
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

In [ ]:
# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)  # Get the class with highest probability

# Print classification report
from sklearn.metrics import classification_report, confusion_matrix
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Sell', 'Hold', 'Buy']))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Display prediction distribution
print("\nPrediction Distribution:")
unique, counts = np.unique(y_pred, return_counts=True)
for value, count in zip(unique, counts):
    class_name = {0: 'Sell', 1: 'Hold', 2: 'Buy'}[value]
    print(f"{class_name} (Class {value}): {count}")

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()

### Evaluation